In [1]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
from lib.object import Object
from lib.functions import get_df
from lib.model import Model
import time
from pathlib import Path
from datetime import datetime

In [2]:

# Создаем наблюдатель
observer_x, observer_y, observer_course, observer_velocity = 0.0, 0.0, 0.0, 5.0
observer = Object(
    "Наблюдатель",
    observer_x,
    observer_y,
    observer_course,
    observer_velocity,
    verbose=True,
)
# Создаем объект
target_bearing, target_distance, target_course, target_velocity = (
    0.0,
    20.0,
    45.0,
    10.0,
)

target = Object(
    "Объект",
    target_bearing,
    target_distance,
    target_course,
    target_velocity,
    observer,
    mode="bdcv",
    verbose=True,
)

# Моделирование траекторий
observer.forward_movement(3 * 60)
observer.change_course(270, "left", omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, "right", omega=0.5)
observer.forward_movement(3 * 60)

# Время движения объекта должно совпадать с временем наблюдателя для TMA
target.forward_movement(len(observer.coords[0]) - 1)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с
Объект движется прямо по курсу 45.0° 1200с


In [3]:
model = Model(observer, target=target, noise_std=np.radians(0.1))

In [4]:
from lib.algorithms import mle_algorithm_v1, mle_algorithm_v2
mle_algorithm_v1(model, [1, 1, 1, 1])

{'ММП scipy': {'Истинные параметры': array([ 0., 20., 45., 10.]),
  'Полученные параметры': array([9.92622058e-04, 1.99619743e+01, 4.49973178e+01, 9.95720043e+00]),
  'Начальное приближение': array([45.        ,  1.41421356, 45.        ,  1.41421356]),
  'Текущие значения': [20.468402895100734,
   27.612210950893108,
   20.45131241465236,
   27.53021703300262],
  'СКО параметров': array([0.00522259, 0.06922209, 0.0449363 , 0.06673826]),
  'Ка, Кб, Кс': [0.0099846619087854, 0.019988702873085747, [1, 1, 1, 1]],
  'Время работы': [0.008104100000000614],
  'Число вычислений функции, число итераций': [51, nan]}}

In [5]:
mle_algorithm_v2(model, [1, 1, 1, 1])

{'ММП': {'Истинные параметры': array([ 0., 20., 45., 10.]),
  'Полученные параметры': array([9.93112584e-04, 1.99619940e+01, 4.49973851e+01, 9.95719571e+00]),
  'Начальное приближение': array([45.        ,  1.41421356, 45.        ,  1.41421356]),
  'Текущие значения': [20.468402895100734,
   27.612210950893108,
   20.451320133252484,
   27.530224530356378],
  'СКО параметров': array([0.00520936, 0.06904265, 0.04481797, 0.06656546]),
  'Ка, Кб, Кс': [0.009984661900939372, 0.019985817156572293, [1, 1, 1, 1]],
  'Время работы': [0.008720999999999535],
  'Число вычислений функции, число итераций': [10, 9]}}

In [29]:
df = get_df(mle_algorithm_v1(model, [1, 1, 1, 1]))
df.drop(0, axis=0, inplace=True)
for i in range(5):
    model.new_target(seed=i)
    df = df.append(get_df(mle_algorithm_v1(model, [1, 1, 1, 1])))
    df = df.append(get_df(mle_algorithm_v2(model, [1, 1, 1, 1])))
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df.round(2))

,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,П0_апр,Д0_апр,К0_апр,V0_апр,Птек_ист,Дтек_ист,Птек_расч,Дтек_расч,СКО X,СКО Y,СКО VX,СКО VY,Ка,Кб,Успех,t,Nf,Iter
0,0.0,29.70,128.73,11.03,359.98,29.81,128.40,11.09,45.0,1.41,45.0,1.41,31.44,22.03,31.46,22.20,0.01,0.13,0.07,0.04,0.01,0.04,"[1, 1, 1, 1]",0.01,72,NaN
0,0.0,29.70,128.73,11.03,359.98,29.81,128.40,11.09,45.0,1.41,45.0,1.41,31.44,22.03,31.46,22.20,0.01,0.13,0.07,0.03,0.01,0.04,"[1, 1, 1, 1]",0.01,25,19
0,0.0,23.77,129.66,5.00,360.00,23.69,129.91,4.98,45.0,1.41,45.0,1.41,18.49,18.26,18.47,18.18,0.01,0.06,0.03,0.05,0.01,0.03,"[1, 1, 1, 1]",0.00,63,NaN
0,0.0,23.77,129.66,5.00,360.00,23.69,129.91,4.98,45.0,1.41,45.0,1.41,18.49,18.26,18.47,18.18,0.01,0.06,0.03,0.05,0.01,0.03,"[1, 1, 1, 1]",0.00,12,10
0,0.0,24.62,4.67,10.50,0.01,24.86,4.79,10.25,45.0,1.41,45.0,1.41,3.63,34.63,3.65,34.57,0.01,0.17,0.02,0.27,0.01,0.07,"[1, 1, 1, 1]",0.00,46,NaN
0,0.0,24.62,4.67,10.50,0.01,24.86,4.79,10.25,45.0,1.41,45.0,1.41,3.63,34.63,3.65,34.57,0.01,0.16,0.02,0.26,0.01,0.07,"[1, 1, 1, 1]",0.00,9,8
0,0.0,29.79,127.47,7.91,0.02,29.64,128.29,7.88,45.0,1.41,45.0,1.41,22.14,23.10,22.12,22.85,0.01,0.12,0.05,0.05,0.01,0.05,"[1, 1, 1, 1]",0.00,72,NaN
0,0.0,29.79,127.47,7.91,0.02,29.64,128.29,7.88,45.0,1.41,45.0,1.41,22.14,23.10,22.12,22.85,0.01,0.11,0.05,0.05,0.01,0.05,"[1, 1, 1, 1]",0.00,17,13
0,0.0,48.52,98.50,14.73,0.00,48.35,98.78,14.64,45.0,1.41,45.0,1.41,23.31,47.13,23.29,46.88,0.01,0.37,0.16,0.10,0.01,0.03,"[1, 1, 1, 1]",0.00,61,NaN
0,0.0,48.52,98.50,14.73,0.00,48.35,98.78,14.64,45.0,1.41,45.0,1.41,23.31,47.13,23.29,46.88,0.01,0.38,0.17,0.10,0.01,0.03,"[1, 1, 1, 1]",0.00,15,13
